In [1]:
import http.client
import json
import csv
import time
import pandas as pd
from datetime import datetime
import requests
#import pandas as pd
#import numpy as np

In [2]:
#Get all the addresses
 
df_address = pd.read_csv("Top590Wallets.csv")    
df_address    

address_list = df_address['Address'].drop_duplicates().to_list()
address_list
#import requests

#response = requests.get('https://api.blockchair.com/bitcoin/addresses?limit=100')

#if response.status_code == 200:
#    content = response.json()

#print(content)

['34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
 '35hK24tcLEWcgNA4JxpvbkNkoAcDGqQPsP',
 'bc1qgdjqv0av3q56jvd82tkdjpy7gdp9ut8tlqmgrpmv24sq90ecnvqqjwvw97',
 '1P5ZEDWTKTFGxQjZphgWPQUpe554WKDfHQ',
 '37XuVSEpWW4trkfmvWzegTHQt7BdktSKUs',
 '38UmuUqPCrFmQo4khkomQwZ4VbY2nZMJ67',
 '3Kzh9qAqVWQhEsfQz7zEQL1EuSx5tyNLNS',
 '1FeexV6bAHb8ybZjqQMjJrcCrHGW9sb6uF',
 'bc1qa5wkgaew2dkv56kfvj49j0av5nml45x9ek9hz6',
 '37tRFZw7n94Jddq6TfVs3MbCXmDX6eMfeY',
 '3JurbUwpsAPqvUkwLM5CtwnEWrNnUKJNoD',
 '1LdRcdxfbSnmCYYNdeYpUnztiYzVfBEQeC',
 '1AC4fMwgY8j9onSbXEWeH6Zan8QGMSdmtA',
 '1LruNZjwamWJXThX2Y8C2d47QqhAkkc5os',
 '385cR5DM96n1HvBDMzLHPYcw89fZAXULJP',
 'bc1qvpgyac88vqtslewxu7yu9dqwp8rd83zch55zpm3xgn3mgg72w3kqv0s8qa',
 '3P5RCQv6k5Gc9WixzU4MAxZ69CsW1eta2q',
 '3FGKwP7XQA9Gb27if7zQGJSSynbzWLrV3p',
 'bc1q5shngj24323nsrmxv99st02na6srekfctt30ch',
 '3D8qAoMkZ8F1b42btt2Mn5TyN7sWfa434A',
 '17hf5H8D6Yc4B7zHEg3orAtKn7Jhme7Adx',
 'bc1qt4h06ssqtwncv5rtf8c8v53emwmvr9j5peq9quj9vf0jzu2egnxqpesxny',
 '12XqeqZRVkBDgmPLVY4ZC6Y4ruUUEug8Fx',
 '3

In [3]:
#address_list = []

#for i in range(0,len(content['data'])):
#    address_list.append(content['data'][i]['address'])


In [4]:
len(address_list)

582

In [5]:
#Retrieving for 100 addresses at a time
address_list = address_list[100:201]
len(address_list)

101

# 17rm2dvb439dZqyMe2d4D6AQJSgg6yeNRn

Use the address here to find hash from tx_id:
https://sochain.com/api/v2/address/BTC/17rm2dvb439dZqyMe2d4D6AQJSgg6yeNRn

Use the tx_id here to find transactions:
https://api.whale-alert.io/v1/transaction/bitcoin/d02b62bce9017db5cbac488a99e4bd298bad37f2728400ca6f8d4685560d96ca?api_key=SRA4UV9DvHHLzsXmga6qooRyp1jiRc3U

In [6]:
from_owner_type = []
#import time
#import pandas as pd
trans_data = pd.DataFrame([])
hash_list = []

for a in range(0, len(address_list)):
    response = requests.get('https://sochain.com/api/v2/address/BTC/{}'.format(address_list[a]))
    #response = requests.get('https://sochain.com/api/v2/address/BTC/17rm2dvb439dZqyMe2d4D6AQJSgg6yeNRn')
    print("a:", a)
    #print(response.status_code)
    if response.status_code == 200:
        
        hash_data = response.json()
        #print(hash_data)
        for b in range(0,len(hash_data['data']['txs'])):
            hash_list.append(hash_data['data']['txs'][b]['txid'])
            hash_id = hash_data['data']['txs'][b]['txid']
            
            #for i in range(0, len(hash_list)):
            #print('hash_id:', hash_id)
            time.sleep(1)
            response = requests.get('https://api.whale-alert.io/v1/transaction/bitcoin/{}?api_key=SRA4UV9DvHHLzsXmga6qooRyp1jiRc3U'.format(hash_id))
            #trans_raw = []
            #print(response.status_code)
            if response.status_code == 200:

                trans_raw = response.json()

                #print (trans_raw)
                if(trans_raw['count'] != 0):
                    #print(trans_raw['transactions'])
                    for j in range (0, len(trans_raw['transactions'])):

                        trans_data = trans_data.append(pd.DataFrame({'address': address_list[a],
                                                                     'hash_id': hash_id, 
                                                                     'from_owner_type': trans_raw['transactions'][j]['from']['owner_type'], 
                                                                     'to_owner_type': trans_raw['transactions'][j]['to']['owner_type'],
                                                                     'timestamp': trans_raw['transactions'][j]['timestamp'],
                                                                     'DateTime' : datetime.fromtimestamp(trans_raw['transactions'][j]['timestamp']),
                                                                     'amount':trans_raw['transactions'][j]['amount'],
                                                                     'amount_usd':trans_raw['transactions'][j]['amount_usd']},
                                                                     index=[0]),ignore_index=True)


                
        #    from_owner_type.append(trans_raw['transactions'][j]['from']['owner_type'])
        #    print(trans_raw['transactions'][j]['from']['owner_type'])
#response = requests.get('https://api.whale-alert.io/v1/transaction/bitcoin/{}?api_key=SRA4UV9DvHHLzsXmga6qooRyp1jiRc3U'.format('8f2b81360fe4542dd4d83fe4e8be5971aec80ef0b747b52a393789a1f0f0ae5b'))

a: 0
a: 1
a: 2
a: 3
a: 4
a: 5
a: 6
a: 7
a: 8
a: 9
a: 10
a: 11
a: 12
a: 13
a: 14
a: 15
a: 16
a: 17
a: 18
a: 19
a: 20
a: 21
a: 22
a: 23
a: 24
a: 25
a: 26
a: 27
a: 28
a: 29
a: 30
a: 31
a: 32
a: 33
a: 34
a: 35
a: 36
a: 37
a: 38
a: 39
a: 40
a: 41
a: 42
a: 43
a: 44
a: 45
a: 46
a: 47
a: 48
a: 49
a: 50
a: 51
a: 52
a: 53
a: 54
a: 55
a: 56
a: 57
a: 58
a: 59
a: 60
a: 61
a: 62
a: 63
a: 64
a: 65
a: 66
a: 67
a: 68
a: 69
a: 70
a: 71
a: 72
a: 73
a: 74
a: 75
a: 76
a: 77
a: 78
a: 79
a: 80
a: 81
a: 82
a: 83
a: 84
a: 85
a: 86
a: 87
a: 88
a: 89
a: 90
a: 91
a: 92
a: 93
a: 94
a: 95
a: 96
a: 97
a: 98
a: 99
a: 100


In [7]:
trans_data.to_csv('raw_transactions_200.csv', sep=',', encoding='utf-8')

#print(trans_raw['transactions'][0]['from']['owner_type'])
